In [2]:
import sys
sys.path.append('../')
import abc
import logging
import mne
import numpy as np
import pandas as pd
from brainModels.datasets.erpCoreP300 import ERPCORENP300
from brainModels.preprocessing.erp import ERP
from brainModels.featureExtraction.features import AutoRegressive
from brainModels.featureExtraction.features import PowerSpectralDensity
from brainModels.featureExtraction.twinNeural import TwinNeuralNetwork
from brainModels.datasets import utils
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from brainModels.evaluations.single_session_open_set import SingleSessionOpenSet
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC                                                                                 
from sklearn.ensemble import RandomForestClassifier
from brainModels.analysis.plotting import Plots 
import os

## Performing Single Session Evaluation for dataset ERPCORENP300

ERPCORENP300 is ERP based dataset and was generated as part of the study [1] E. S. Kappenman, J. L. Farrens, W. Zhang, A. X. Stewart, S. J. Luck, Erp core: An open resource for human event-related potential research, NeuroImage 225 (2021) 117465. 

| Name      | #Subj | #Chan | #Trials/class | Trials length | Sampling Rate | #Sessions |
|-----------|-------|-------|---------------|---------------|---------------|-----------|
| ERP: P300 | 40    | 32    | 5 NT x 1 T    | 1s            | 1024Hz        | 1         |

**Description of ERP Task in ERPCOREP300**

This ERP task Utilized an active 
visual oddball task to evoke the P300 response. Randomly presented letters A, B, C, D, and E
with equal probabilities (p = .2 for each letter). Designated one letter as the target 
for a block, and participants identified whether each stimulus was the target or a 
non-target. This approach maintained a constant target probability of .2, with the 
same physical stimulus serving as a target in some blocks and a non-target in others..

### Creating instances of datasets and ERP with default parameters

Following are the default parameters of dataset and paradigm ERP:</br></br>
    <i>Number of Subjects=40</i></br>
    <i>Sample_duration=1 seconds (-200,800)</i></br>
    <i>Baseline_Correction=True</i>

In [3]:
dataset1=ERPCORENP300()
paradigm=ERP()

### Intializing pipelines for 4 shallow Classifiers i.e., LDA, LR, KNN and RF with AR Parameters and PSD Features with default Parameters

Following are the default parameters of AR coeffecients and Twin Neural Networks:</br></br>
    <i>AR Order=1</i></br>
    <i>Batch_Size=192</i></br>
    <i>Epochs=100</i></br>
    

In [4]:
pipeline={}
pipeline['AR+PSD+LDA']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), LDA())
pipeline['AR+PSD+LR']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), LogisticRegression())
pipeline['AR+PSD+KNN']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), KNeighborsClassifier())
pipeline['AR+PSD+RF']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), RandomForestClassifier())
pipeline['TNN']=make_pipeline(TwinNeuralNetwork())

### Creating Authentication pipeline for Single session evaluation under unkown attacker Scenario

In [5]:
evaluation=SingleSessionOpenSet(paradigm=paradigm, datasets=dataset1, overwrite=False)

### Executing the Single Session Authentication Pipeline

In [6]:
results=evaluation.process(pipeline)

### Statistical Analysis of the performace of Single Session Evalaution Across the two datasets

Comparative Analysis of Average EER and FNMR at 1%, 0.1%, and 0.01% FMR thresholds across single-session dataset. The evaluation encompasses single-session evaluation Scheme, with a focus on classifiers’ performance in the unknown attacker scenario.

In [ ]:
single_session_df=results.groupby(['eval Type','dataset','pipeline']).agg({
                'eer': lambda x: f'{np.mean(x)*100:.3f} ± {np.std(x)*100:.3f}',
                'frr_1_far': lambda x: f'{np.mean(x)*100:.3f}',
                'frr_0.1_far': lambda x: f'{np.mean(x)*100:.3f}',
                'frr_0.01_far': lambda x: f'{np.mean(x)*100:.3f}'    
            }).reset_index()

display(single_session_df)